In [4]:
# Setups
import pandas as pd
import plotly.express as px

data_location = "datasets/"

baseball_players = pd.read_csv(data_location + "baseball_players.csv")
hitting_stats = pd.read_csv(data_location + "exit_velocity.csv")

#print(hitting_stats)

hello awpirugbwaiuwav ape9uechbaepuvhbseviusbev

no u